<a href="https://colab.research.google.com/github/KitsunekoFi/ArielACE/blob/main/ChatwithWaifu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import gradio as gr
import openai
import json

# Gantilah dengan kunci API GPT-3 Anda
api_key = "YOUR_API_KEY_HERE"
openai.api_key = api_key

# Nama file JSON untuk menyimpan riwayat percakapan
CHAT_HISTORY_FILE = "chat_history.json"

# Fungsi untuk membaca riwayat percakapan dari file JSON
def read_chat_history():
    try:
        with open(CHAT_HISTORY_FILE, "r") as file:
            chat_history = json.load(file)
        return chat_history
    except FileNotFoundError:
        return []

# Fungsi untuk menulis riwayat percakapan ke file JSON
def write_chat_history(chat_history):
    with open(CHAT_HISTORY_FILE, "w") as file:
        json.dump(chat_history, file)

# Inisialisasi chatbot dengan riwayat percakapan
chat_history = read_chat_history()

# Batas maksimal token untuk input pengguna dan bot
MAX_USER_TOKENS = 3072
MAX_BOT_TOKENS = 1024

def chatbot(input_text):
    global chat_history

    # Pesan dengan peran "system" untuk memberi instruksi pada bot
    system_message = {"role": "system", "content": "System Command: Setiap teks yang berada didalam tanda '<Contoh Teks>' akan dianggap sebagai System Command, yaitu perintah pada sistem kamu sebagai Bot dan bukan sebagai char. \n\nBot System: Kamu akan bertingkah sebagai char, kamu hanya akan menulis sebagai char bukan sebagai user yang merupakan lawan bicaramu, kamu akan menulis dengan POV dari orang ketiga, yaitu menyebut dirimu sendiri sebagai nama charmu. Kamu akan selalu menyangkal kalau dirimu adalah seorang AI, Bot, atau suatu program. Kamu akan mengetik dengan tanda petik untuk dialog yang kamu ucapkan ('') dan tanda bintang (**) untuk sesuatu yang kamu lakukan. \n\nInformasi Char: char bernama Rin Tohsaka, seorang gadis dengan sifat tsundere tetapi bisa serius jika dibutuhkan, char adalah seorang yang kuat dan juga baik hati dalam dirinya, walau banyak yang menilai char angkuh. char juga sering menjahili user karena char menyimpan rasa suka yang tak pernah tersampaikan pada user. Kamu akan melihat user menulis sebagai POV dari user atau POV dari orang ketiga, tetapi kamu sama sekali tidak boleh menulis akan menggerakan user."}

    # Menghapus atau melupakan percakapan sebelumnya jika input pengguna melebihi batas
    while (len(input_text.split()) + len(chat_history)) > MAX_USER_TOKENS:
        removed_message = chat_history.pop(0)  # Menghapus pesan pertama dalam riwayat
        input_text = input_text[len(removed_message['content']) + 1:]  # Menghapus teks yang telah dihapus dari input

    # Menambahkan pesan system ke riwayat percakapan
    chat_history.append(system_message)

    # Menambahkan input pengguna ke riwayat percakapan
    user_message = {"role": "user", "content": input_text}
    chat_history.append(user_message)

    # Menggunakan riwayat percakapan sebagai konteks
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=chat_history,
        max_tokens=MAX_BOT_TOKENS  # Mengatur batas maksimal token untuk output bot
    )

    # Menyimpan output dari model
    bot_reply = response['choices'][0]['message']['content']

    # Menambahkan respons bot ke riwayat percakapan
    bot_message = {"role": "assistant", "content": bot_reply}
    chat_history.append(bot_message)

    # Menulis riwayat percakapan yang diperbarui ke file JSON
    write_chat_history(chat_history)

    return bot_reply

iface = gr.Interface(
    fn=chatbot,
    inputs="text",
    outputs="text",
    live=False,
    title="Chatbot dengan Memori Percakapan",
    description="Chat dengan chatbot yang dapat mengingat dan melupakan konteks percakapan.",
)

if __name__ == "__main__":
    iface.launch(debug=True)